In [1]:
# cv=5
# print(cross_val_score(best_model, X_trainval, y_trainval.values.ravel(), cv=5, scoring="accuracy").mean())
# 0.8116666666666668

# cv=10
# print(cross_val_score(best_model, X_trainval, y_trainval.values.ravel(), cv=10, scoring="accuracy").mean())
# 0.8089999999999999
# LB
# 0.7985000

In [2]:
# # signate CLIのインストール
# !pip install signate

# # APIトークンをダウンロード
# !signate token --email="" --password=""

# # 現在開催中のコンペリストを表示
# # !signate list

# # 上のリストのIDでコンペを指定して、データをダウンロード
# !signate download --competition-id=748 #【第32回_Beginner限定コンペ】診断データを使った糖尿病発症予測

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=8721978aa21a6eba5a7a78174043a4bf41e5b6567bab5b21f0579818b6ccb734
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
The API Token has been downloaded successfully.
sample_submit.csv

test.csv

train.csv


Download completed.


In [3]:
import pandas as pd

train = pd.read_csv("./train.csv", index_col = 0)
test = pd.read_csv("./test.csv", index_col = 0)

# 内容確認
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000 entries, 200 to 2732
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               3000 non-null   int64  
 1   Glucose                   3000 non-null   int64  
 2   BloodPressure             3000 non-null   int64  
 3   SkinThickness             3000 non-null   int64  
 4   Insulin                   3000 non-null   int64  
 5   BMI                       3000 non-null   float64
 6   DiabetesPedigreeFunction  3000 non-null   float64
 7   Age                       3000 non-null   int64  
 8   Outcome                   3000 non-null   int64  
dtypes: float64(2), int64(7)
memory usage: 234.4 KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 398 to 361
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnanc

In [4]:
import numpy as np

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

import lightgbm as lgb

!pip install optuna==2.9.0
import optuna
import optuna.integration.lightgbm as lgb_o

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 KB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.5/210.5 KB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.2/147.2 KB 19.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 KB 16.0 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=571b1d1888b76a88c904705c1643bf21685c2ec90bd1963f0af003f302f0eeda
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip


In [5]:
X_trainval = train.drop('Outcome', axis=1)
y_trainval = train[['Outcome']].values.ravel()

# 交差検証分割数
cv = 10

# LightGBM用のデータセットに変換
trainval = lgb.Dataset(X_trainval, y_trainval)

# ハイパーパラメータサーチ&モデル構築
params = {'objective': 'binary',
          'metric': "binary_logloss",
          'random_seed':0} 

# クロスバリデーションによるハイパーパラメータの探索 3fold
tuner = lgb_o.LightGBMTunerCV(params, trainval,
                              verbose_eval=100,
                              early_stopping_rounds=100,
                              folds=KFold(n_splits=cv),
                              optuna_seed=0)

# ハイパーパラメータ探索の実行
tuner.run()

[I 2023-03-05 15:34:40,432] A new study created in memory with name: no-name-16d10b49-5b6c-47ee-b2e4-4a662ca75c48
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

[100]	cv_agg's binary_logloss: 0.428419 + 0.0354098


feature_fraction, val_score: 0.421331:  14%|#4        | 1/7 [00:03<00:19,  3.24s/it]

[100]	cv_agg's binary_logloss: 0.439576 + 0.0244077


feature_fraction, val_score: 0.421331:  29%|##8       | 2/7 [00:04<00:09,  1.94s/it]

...

[100]	cv_agg's binary_logloss: 0.421881 + 0.0277911


min_data_in_leaf, val_score: 0.412723:  80%|########  | 4/5 [00:05<00:01,  1.47s/it]

[100]	cv_agg's binary_logloss: 0.418581 + 0.0263499


min_data_in_leaf, val_score: 0.412723: 100%|##########| 5/5 [00:07<00:00,  1.41s/it]

[200]	cv_agg's binary_logloss: 0.430491 + 0.0300472


In [6]:
# サーチしたパラメータ
best_params = tuner.best_params
best_model = lgb.LGBMClassifier(**best_params)
best_model.fit(X_trainval, y_trainval)

LGBMClassifier(bagging_fraction=1.0, bagging_freq=0, feature_fraction=0.4,
               feature_pre_filter=False, lambda_l1=0.0026599310838681858,
               lambda_l2=0.0008015832747965142, metric='binary_logloss',
               num_leaves=22, objective='binary', random_seed=0)

In [7]:
print(cross_val_score(best_model, X_trainval, y_trainval, cv=cv, scoring="accuracy").mean())

0.8089999999999999


In [8]:
# 予測
pred = best_model.predict(test)

In [9]:
submit = pd.read_csv("./sample_submit.csv", index_col=0, header=None)

In [10]:
submit[1] = pred
submit.to_csv("./submit.csv", header=False)
submit

,1
0,
398,0
3833,0
4836,0
4572,1
636,0
...,...
3138,0
191,0
3294,0


In [11]:
# ファイルを提出 提出先のコンペのIDを指定する
!signate submit --competition-id=748 ./submit.csv

You have successfully submitted your predictions.We will send you the submission result to your email address.
